In [1]:
import geopandas as gpd
import rasterio
import os
import numpy as np
from rasterio import features
from rasterio.features import geometry_mask

# Define the paths to your shapefile and TIFF directory
shp_file = "/home/karim/WaPOR/data/algeria/segmentation/24tiles/V3/SNIC_30000_V2.shp"
tif_directory = "/home/karim/WaPOR/data/algeria/outputs/PHENOL/"
output_file = "/home/karim/WaPOR/data/algeria/outputs/cumul/segments_rainfall_season1_2019.tif"

# Years of the Growing Season:
ny1, ny2, ny3 = 18, 19, 20

# Load the shapefile
gdf = gpd.read_file(shp_file)
array_length = gdf.shape[0]

# Preallocate arrays
start_array = np.full(array_length, np.nan, dtype=int)
end_array = np.full(array_length, np.nan, dtype=int)
eta_all = np.full(array_length, -99., dtype=float)

# Open the model GeoTIFF to get its projection and resolution
model = "/home/karim/WaPOR/data/algeria/segmentation/24tiles/V3/SNIC_30000_V2.tif"
with rasterio.open(model) as model_ds:
    model_crs = model_ds.crs
    model_transform = model_ds.transform
    model_width = model_ds.width
    model_height = model_ds.height

# Open the output GeoTIFF file for writing
accumulated_values = np.zeros((model_height, model_width), dtype='float32')
def get_month_and_dekade(k):
    # Ensure k is within the valid range
    if k < 1 or k > 36:
        if k == 0:
            return '00', 'D0'
        else:
            raise ValueError("k must be between 1 and 36")
    # Calculate the month and dekade
    month_index = (k - 1) // 3
    dekade = (k - 1) % 3 + 1
    # Convert month_index to a month number (1-based)
    month = month_index + 1
    # Format month as a two-digit string
    month_str = f"{month:02d}"
    dd='D' + str(dekade)

    return month_str, dd

# Iterate over the shapefile features
for idx, row in gdf.iterrows():
    segment_id = row['DN']

    # Check if the TIFF files exist
    tif_file_path = os.path.join(tif_directory, f"start_final_season_2019.tif")
    tif_file_path2 = os.path.join(tif_directory, f"end_final_season_2019.tif")

    with rasterio.open(tif_file_path) as src1, rasterio.open(tif_file_path2) as src2:
        geom = row['geometry']

        # Create a mask for the segment
        mask = features.geometry_mask([geom], out_shape=src1.shape, transform=src1.transform, invert=True)

        # Read the values from the TIFF file using the mask
        values1 = src1.read(1, masked=True)
        values2 = src2.read(1, masked=True)

        segment_values1 = values1[mask]
        segment_values2 = values2[mask]

        # Check if segment_values1 has a size greater than 0 before accessing its elements
        if len(segment_values1) > 0:
            start_array[idx] = np.asarray(segment_values1[0])
        else:
            start_array[idx] = 0

        # Check if segment_values2 has a size greater than 0 before accessing its elements
        if len(segment_values2) > 0:
            end_array[idx] = np.asarray(segment_values2[0])
        else:
            end_array[idx] = 0

        # Process ETA data
        eta = np.zeros(mask.sum(), dtype=float)  # Initialize eta with the correct shape
        nmfic = []
        nmyy = []

        n1 = start_array[idx]
        n2 = end_array[idx]
        if n1 < 0. and n2 < 0.:
            n1 = 0
            n2 = 0
            start_array[idx] = 0
            end_array[idx]   = 0
        
        for i in range(n1, n2 + 1):
            if i < 37:
                ssy, k = str(ny1).zfill(2), i - 0
            elif 36 < i < 73:
                ssy, k = str(ny2).zfill(2), i - 36
            elif 72 < i < 109:
                ssy, k = str(ny3).zfill(2), i - 72
            else:
                k, ssy = 0, str(0).zfill(2)
            newssy = '20' + ssy
            #print('k = ',k, ' i ',i, 'n1 = ',n1,' n2 = ',n2)
            newmm, newD = get_month_and_dekade(k)
            titr=newssy + '-' + newmm + '-' + newD
            nmfic.append(f"{k:02d}")
            #nmfic.append(f"{titr}")
            nmyy.append(newssy)


        if len(nmfic) > 1 and k > 0:
            for j in range(len(nmfic)):
                input_raw_file = f'/home/karim/WaPOR/data/algeria/outputs/chirps/AVG_50k_{nmyy[j]}{nmfic[j]}.tif'
                with rasterio.open(input_raw_file) as src:
                    values1 = src.read(1, masked=True)

                    # Check if the shapes are compatible
                    if values1[mask].shape == eta.shape:
                        eta += values1[mask]
                    else:
                        # Handle the case where shapes are not compatible (adjust as needed)
                        print(f"Shapes are not compatible for segment {segment_id}")

            if eta.shape[0] > 0 and not np.isnan(eta[0]):
                eta_all[idx] = float(eta[0])
                print('Total ETA For This Segment ', idx, ' = ', eta_all[idx])
            else:
                eta_all[idx] = -99.
                print('          ', 'No season for this segment ! ', idx)

    # Process accumulated values
    segment_gdf = gdf[gdf['DN'] == segment_id]
    segment_geometry = segment_gdf.unary_union
    segment_shapes = [(segment_geometry, 1)]
    segment_mask = features.geometry_mask(
        segment_shapes,
        out_shape=(model_height, model_width),
        transform=model_transform,
        invert=True
    )
    accumulated_values += segment_mask * np.nan_to_num(eta_all[idx], nan=-99.)

# Open the output GeoTIFF file for writing
with rasterio.open(output_file, 'w', driver='GTiff', height=model_height, width=model_width, count=1,
                   dtype='float32', crs=model_crs, transform=model_transform) as dst:
    dst.write(accumulated_values, 1)

print(f"Saved segments with values to {output_file}")

/home/karim/miniconda3/lib/python3.12/site-packages/numpy/core/numeric.py:330: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')


Total ETA For This Segment  307  =  269.9823942184448
Total ETA For This Segment  308  =  12.902202606201172
Total ETA For This Segment  309  =  269.9823942184448
Total ETA For This Segment  310  =  269.9823942184448
Total ETA For This Segment  311  =  78.76014161109924
Total ETA For This Segment  312  =  257.5418314933777
Total ETA For This Segment  313  =  431.30539751052856
Total ETA For This Segment  314  =  6.893699645996094
Total ETA For This Segment  315  =  6.893699645996094
Total ETA For This Segment  316  =  72.75163865089417
Total ETA For This Segment  317  =  257.5418314933777
Total ETA For This Segment  318  =  56.115623474121094
Total ETA For This Segment  319  =  269.9823942184448
Total ETA For This Segment  320  =  131.24252557754517
Total ETA For This Segment  321  =  6.893699645996094
Total ETA For This Segment  322  =  76.32030200958252
Total ETA For This Segment  323  =  76.32030200958252
Total ETA For This Segment  324  =  257.5418314933777
Total ETA For This Segme

KeyboardInterrupt: 